In [3]:
import pandas as pd

df = pd.read_csv("hu_r_0.csv")
df.head()


,Hand,RAISE 25,FOLD,RAISE 25 EV,FOLD EV
0,3c2c,99.999994,0.000006,3.470988,0.000005
1,3c2d,0.000000,100.000000,-17.809067,0.000000
2,3c2h,0.000000,100.000000,-17.809067,0.000000
3,3c2s,0.000000,100.000000,-17.809067,0.000000
4,4c2c,99.999994,0.000006,10.348983,0.000005


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Load data
df = pd.read_csv("hu_r_0.csv")

# 2. Create a function to parse the 'Hand' column into numeric features
rank_map = {'2':2, '3':3, '4':4, '5':5, '6':6,
            '7':7, '8':8, '9':9, 'T':10,
            'J':11, 'Q':12, 'K':13, 'A':14}
suit_map = {'c':1, 'd':2, 'h':3, 's':4}

def parse_hand(hand_str):
    # Example: '3c2c' -> card1='3c', card2='2c'
    card1 = hand_str[0:2]
    card2 = hand_str[2:4]
    
    r1 = rank_map[card1[:-1]]
    s1 = suit_map[card1[-1]]
    r2 = rank_map[card2[:-1]]
    s2 = suit_map[card2[-1]]
    return r1, s1, r2, s2

df[['rank1', 'suit1', 'rank2', 'suit2']] = df['Hand'].apply(
    lambda h: pd.Series(parse_hand(h))
)

# 3. Encode position (SB=0, BB=1) if you have a 'Position' column
#position_map = {'SB': 0, 'BB': 1}
#df['position'] = df['Position'].map(position_map)

# 4. Create a label: for simplicity, pick the highest-probability action
def get_label(row):
    # Suppose columns are 'RAISE 25' and 'FOLD'. Use the one with higher probability
    if row['RAISE 25 EV'] > row['FOLD EV']:
        return 1  # raise
    else:
        return 0  # fold

df['action_label'] = df.apply(get_label, axis=1)

# 5. Prepare features and labels
feature_cols = ['rank1', 'suit1', 'rank2', 'suit2']
X = df[feature_cols]
y = df['action_label']

# 6. Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 7. Train a RandomForest model
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

# 8. Evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Test Accuracy:", acc)
print("Confusion Matrix:\n", cm)


Test Accuracy: 0.9849624060150376
Confusion Matrix:
 [[ 43   2]
 [  2 219]]


In [26]:

# Construct the row as a 2D array: one row with all features
custom_hand = np.array([[3, 1, 2, 1]])  # SB with AcAd

# Predict the class (0=fold, 1=raise)
prediction = model.predict(custom_hand)
# If you also want predicted probabilities for each class:
prediction_proba = model.predict_proba(custom_hand)

print("Predicted label:", prediction)  # 0 or 1
print("Predicted probabilities:", prediction_proba) 


Predicted label: [1]
Predicted probabilities: [[0.143 0.857]]


d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [23]:
# 1. Get the predictions
y_pred = model.predict(X_test)

# 2. Combine them with the actual test labels and the original data
test_results = X_test.copy()
test_results['Actual_Label'] = y_test.values
test_results['Predicted_Label'] = y_pred

# 3. Print or inspect a few rows
print(test_results.head(10))


      rank1  suit1  rank2  suit2  Actual_Label  Predicted_Label
889      13      3     12      1             1                1
467      11      2      4      1             0                0
887      13      3     10      1             1                1
175      11      1      6      4             1                1
1286     14      4     13      1             1                1
479      11      2      6      2             1                1
275      14      1     13      1             1                1
1244     13      4      6      2             1                1
405       9      2      4      1             0                0
289      14      1      3      3             1                1


Test Accuracy: 0.9962311557788944
Confusion Matrix:
 [[155   3]
 [  0 638]]
Stack size 80 -> Accuracy: 0.9894736842105263
Stack size 100 -> Accuracy: 1.0
Stack size 120 -> Accuracy: 1.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Load CSVs for each stack size
df_80 = pd.read_csv("hu_80bb_r_0.csv")
df_100 = pd.read_csv("hu_100bb_r_0.csv")
df_120 = pd.read_csv("hu_120bb_r_0.csv")

# 2. Add stack_size column
df_80["stack_size"] = 80
df_100["stack_size"] = 100
df_120["stack_size"] = 120

# 3. Merge
df = pd.concat([df_80, df_100, df_120], ignore_index=True)

# 4. Create label (0=fold, 1=raise) from solver outputs
def get_label(row):
    if row["RAISE 25"] > row["FOLD"]:
        return 1
    else:
        return 0

df["action_label"] = df.apply(get_label, axis=1)

# 5. Parse the hole cards into numeric features
rank_map = {'2':2, '3':3, '4':4, '5':5, '6':6,
            '7':7, '8':8, '9':9, 'T':10,
            'J':11, 'Q':12, 'K':13, 'A':14}
suit_map = {'c':1, 'd':2, 'h':3, 's':4}

def parse_hand(hand_str):
    card1 = hand_str[0:2]
    card2 = hand_str[2:4]
    
    r1 = rank_map[card1[:-1]]
    s1 = suit_map[card1[-1]]
    r2 = rank_map[card2[:-1]]
    s2 = suit_map[card2[-1]]
    return r1, s1, r2, s2

df[["rank1", "suit1", "rank2", "suit2"]] = df["Hand"].apply(
    lambda h: pd.Series(parse_hand(h))
)

# 6. Create suit/pair/connector indicators
df["is_suited"] = (df["suit1"] == df["suit2"]).astype(int)
df["is_pair"]   = (df["rank1"] == df["rank2"]).astype(int)

def is_connector(row):
    return 1 if abs(row["rank1"] - row["rank2"]) == 1 else 0

df["is_connector"] = df.apply(is_connector, axis=1)

# 7. Define features and label
feature_cols = ["rank1", "suit1", "rank2", "suit2",
                "stack_size",
                "is_suited", "is_pair", "is_connector"]
X = df[feature_cols]
y = df["action_label"]

# 8. Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 9. Train model
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

# 10. Evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Test Accuracy:", acc)
print("Confusion Matrix:\n", cm)

# Optional: check accuracy per stack size
X_test_eval = X_test.copy()
X_test_eval["true"] = y_test.values
X_test_eval["pred"] = y_pred

for s in [80, 100, 120]:
    subset = X_test_eval[X_test_eval["stack_size"] == s]
    acc_s = accuracy_score(subset["true"], subset["pred"])
    print(f"Stack size {s} -> Accuracy: {acc_s}")


In [45]:
def prepare_features(hand_str, stack_size):
    """
    Convert a hand like 'AcAd' and a stack size into the feature vector:
    [rank1, suit1, rank2, suit2, stack_size, is_suited, is_pair, is_connector].
    Returns a 2D array suitable for model.predict().
    """
    # 1. Parse hole cards
    #    For a 4-char string like "AcAd":
    #       card1 = "Ac"
    #       card2 = "Ad"
    card1 = hand_str[0:2]
    card2 = hand_str[2:4]

    #    Extract rank & suit
    r1 = rank_map[card1[:-1]]
    s1 = suit_map[card1[-1]]
    r2 = rank_map[card2[:-1]]
    s2 = suit_map[card2[-1]]

    # 2. Additional indicators
    is_suited = 1 if (s1 == s2) else 0
    is_pair = 1 if (r1 == r2) else 0
    is_connector = 1 if abs(r1 - r2) == 1 else 0

    # 3. Create the feature row
    features = [
        r1,         # rank1
        s1,         # suit1
        r2,         # suit2
        s2,         # suit2
        stack_size, # the chosen stack size
        is_suited,
        is_pair,
        is_connector
    ]
    
    # Return as a 2D array (one row, many columns)
    return np.array([features])

# Example 1: Pocket Aces suited in clubs and diamonds with an 80bb stack
hand_str = "9c5d"
stack_size = 80
X_custom = prepare_features(hand_str, stack_size)

prediction = model.predict(X_custom)
prediction_proba = model.predict_proba(X_custom)

print("Hand:", hand_str, "Stack:", stack_size)
print("Predicted Action:", prediction[0])        # 0 = fold, 1 = raise
print("Predicted Probabilities:", prediction_proba[0])


Hand: 9c5d Stack: 80
Predicted Action: 1
Predicted Probabilities: [0.002 0.998]


d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [48]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

# -------------------------
# 1. Load Data
# -------------------------
#df = pd.read_csv("hu_sbfirst.csv")  # Replace with your CSV filename


# 1. Load CSVs for each stack size
df_80 = pd.read_csv("hu_80bb_r_0.csv")
df_100 = pd.read_csv("hu_100bb_r_0.csv")
df_120 = pd.read_csv("hu_120bb_r_0.csv")

# 2. Add stack_size column
df_80["stack_size"] = 80
df_100["stack_size"] = 100
df_120["stack_size"] = 120

# 3. Merge
df = pd.concat([df_80, df_100, df_120], ignore_index=True)

# Example columns: Hand,RAISE 25,FOLD
# Some solver frequencies might be extremely close to 100 or 0.

# -------------------------
# 2. Clean Frequencies
# -------------------------
# We'll define a small function that sets frequencies close to 100% → exactly 100,
# and frequencies close to 0% → exactly 0
def fix_freq(freq, eps=1.0):
    """
    If freq is in [100 - eps, 100], set it to 100.
    If freq is in [0, eps], set it to 0.
    Otherwise, leave it as is.
    """
    if freq - eps >= 98.0:
        return 100.0
    elif freq < eps:
        return 0.0
    else:
        return freq

df["RAISE 25"] = df["RAISE 25"].apply(fix_freq)
df["FOLD"] = df["FOLD"].apply(fix_freq)

# Check a few rows
# print(df.head(20))

# -------------------------
# 3. Parse Hole Cards into Features
# -------------------------
rank_map = {'2':2, '3':3, '4':4, '5':5, '6':6,
            '7':7, '8':8, '9':9, 'T':10,
            'J':11, 'Q':12, 'K':13, 'A':14}
suit_map = {'c':1, 'd':2, 'h':3, 's':4}

def parse_hand(hand_str):
    """
    For a 4-char hand, e.g. '3c2c', split into card1='3c', card2='2c'
    Return (rank1, suit1, rank2, suit2)
    """
    card1 = hand_str[0:2]  # e.g. '3c'
    card2 = hand_str[2:4]  # e.g. '2c'
    
    r1 = rank_map[card1[:-1]]
    s1 = suit_map[card1[-1]]
    r2 = rank_map[card2[:-1]]
    s2 = suit_map[card2[-1]]
    return r1, s1, r2, s2

df[["rank1", "suit1", "rank2", "suit2"]] = df["Hand"].apply(
    lambda h: pd.Series(parse_hand(h))
)

# Add extra indicators
df["is_suited"] = (df["suit1"] == df["suit2"]).astype(int)
df["is_pair"]   = (df["rank1"] == df["rank2"]).astype(int)

def is_connector(row):
    return 1 if abs(row["rank1"] - row["rank2"]) == 1 else 0

def is_1_gap(row):
    return 1 if abs(row["rank1"] - row["rank2"]) == 2 else 0

df["is_connector"] = df.apply(is_connector, axis=1)
df["is_1_gap"] = df.apply(is_1_gap, axis=1)

# -------------------------
# 4. (Optional) Add Stack Size
# -------------------------
# If you have a single stack size for all, you can just fix a constant, or
# if your CSV doesn't have stack_size, you can define it manually. For example:
#df["stack_size"] = 100  # or some default
# If your solver data has multiple stack sizes, you'd read them from CSV or do concatenation.

# -------------------------
# 5. Build the Target: Fold Frequency in [0,1]
# -------------------------
# The solver frequencies might be in [0, 100]. Let's convert them to fraction in [0,1].
df["fold_freq"] = df["FOLD"] / 100.0
# (Similarly, raise_freq = df["RAISE 25"] / 100.0, but we only need one if we have exactly two actions.)

# -------------------------
# 6. Define X (Features) and y (Target)
# -------------------------
feature_cols = [
    "rank1", "suit1", "rank2", "suit2",
    "stack_size",
    "is_suited", "is_pair", "is_connector", "is_1_gap"
]
X = df[feature_cols]
y = df["fold_freq"]  # We'll predict the fold frequency

# -------------------------
# 7. K-Fold Cross-Validation for More Robust Performance
# -------------------------
# We'll measure negative mean squared error (neg MSE),
# then convert to positive MSE, and also compute RMSE.

from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)

mse_list = []
for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mse_list.append(mse)

# Evaluate
mse_array = np.array(mse_list)
rmse_array = np.sqrt(mse_array)
print("MSE (per fold):", mse_array)
print("RMSE (per fold):", rmse_array)
print("Mean RMSE:", rmse_array.mean(), "Std dev:", rmse_array.std())

# -------------------------
# 8. Train Final Model on All Data
# -------------------------
# If you want a final model to use (predict probabilities),
# you can retrain on the entire dataset:
final_model = RandomForestRegressor(n_estimators=100, random_state=42)
final_model.fit(X, y)

# Now final_model.predict(...) can be used for new hands.
# e.g., final_model.predict([ [rank1, suit1, rank2, suit2, stack_size, is_suited, is_pair, is_connector] ])


MSE (per fold): [2.62084034e-06 1.19277635e-07 1.97807870e-08 6.79338551e-08
 1.34332828e-07]
RMSE (per fold): [0.0016189  0.00034537 0.00014064 0.00026064 0.00036651]
Mean RMSE: 0.0005464133530296848 Std dev: 0.00054209366008017


RandomForestRegressor(random_state=42)

In [69]:
import numpy as np

rank_map = {'2':2, '3':3, '4':4, '5':5, '6':6,
            '7':7, '8':8, '9':9, 'T':10,
            'J':11, 'Q':12, 'K':13, 'A':14}
suit_map = {'c':1, 'd':2, 'h':3, 's':4}

def prepare_features(hand_str, stack_size):
    """
    Convert a hand like 'AcAd' and a stack size into the feature vector:
    [rank1, suit1, rank2, suit2, stack_size, is_suited, is_pair, is_connector].
    Returns a 2D array suitable for model.predict().
    """
    # Split into two cards
    card1 = hand_str[0:2]  # 'Ac'
    card2 = hand_str[2:4]  # 'Ad'
    
    # Parse ranks and suits
    r1 = rank_map[card1[:-1]]  # 'A' -> 14
    s1 = suit_map[card1[-1]]   # 'c' -> 1
    r2 = rank_map[card2[:-1]]  # 'A' -> 14
    s2 = suit_map[card2[-1]]   # 'd' -> 2

    # Indicators
    is_suited = 1 if s1 == s2 else 0
    is_pair   = 1 if r1 == r2 else 0
    is_connector = 1 if abs(r1 - r2) == 1 else 0
    is_1_gap = 1 if abs(r1 - r2) == 2 else 0

    # Build the row
    features = [
        r1, s1,
        r2, s2,
        stack_size,
        is_suited,
        is_pair,
        is_connector,
        is_1_gap
    ]
    return np.array([features])  # 2D array

# Example usage:
X_custom = prepare_features("2sQd", 100)
pred_fold_freq = final_model.predict(X_custom)[0]  # a single float in [0,1]
pred_raise_freq = 1.0 - pred_fold_freq

#print("Hand = AcAd, Stack = 100bb")
print(f"Predicted fold frequency:  {pred_fold_freq:.4f} ({pred_fold_freq * 100:.2f}%)")
print(f"Predicted raise frequency: {pred_raise_freq:.4f} ({pred_raise_freq * 100:.2f}%)")


Predicted fold frequency:  0.0000 (0.00%)
Predicted raise frequency: 1.0000 (100.00%)


d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [72]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# -------------------------
# 1. Load & Merge Data for Multiple Stack Sizes
# -------------------------
df_80 = pd.read_csv("hu_80bb_r_0.csv")
df_100 = pd.read_csv("hu_100bb_r_0.csv")
df_120 = pd.read_csv("hu_120bb_r_0.csv")

df_80["stack_size"] = 80
df_100["stack_size"] = 100
df_120["stack_size"] = 120

df = pd.concat([df_80, df_100, df_120], ignore_index=True)

# -------------------------
# 2. Clean Frequencies
# -------------------------
# We'll define a small function that sets frequencies near 0 → 0, near 100 → 100.
def fix_freq(freq, eps=1.0):
    """
    If freq >= 100 - eps, set it to 100.
    If freq <= eps,       set it to 0.
    Otherwise, leave it as is.
    """
    if freq >= 100 - eps:
        return 100.0
    elif freq <= eps:
        return 0.0
    else:
        return freq

df["RAISE 25"] = df["RAISE 25"].apply(fix_freq)
df["FOLD"] = df["FOLD"].apply(fix_freq)

# -------------------------
# 3. Parse & Canonicalize Hole Cards
# -------------------------
rank_map = {'2':2, '3':3, '4':4, '5':5, '6':6,
            '7':7, '8':8, '9':9, 'T':10,
            'J':11, 'Q':12, 'K':13, 'A':14}
suit_map = {'c':1, 'd':2, 'h':3, 's':4}

def parse_hand_to_canonical(hand_str):
    """
    hand_str like 'Qd2s' or '2sQd' (4 chars total).
    1) Extract card1, card2
    2) Convert each to (rank, suit)
    3) Canonicalize: Ensure (rank1, suit1) >= (rank2, suit2) 
       by rank primarily, then suit as tiebreaker
    4) Return (rank1, suit1, rank2, suit2)
    """
    card1 = hand_str[0:2]  # e.g. 'Qd'
    card2 = hand_str[2:4]  # e.g. '2s'
    
    # Parse ranks and suits
    r1 = rank_map[card1[:-1]]
    s1 = suit_map[card1[-1]]
    r2 = rank_map[card2[:-1]]
    s2 = suit_map[card2[-1]]
    
    # If second card is "bigger" by rank or tie rank & bigger suit,
    # swap so that (r1, s1) is always the "higher" or canonical card.
    # This ensures Qd2s == 2sQd => same final representation.
    if (r2 > r1) or (r2 == r1 and s2 > s1):
        r1, r2 = r2, r1
        s1, s2 = s2, s1
    
    return r1, s1, r2, s2

# Apply to entire DataFrame
df[["rank1", "suit1", "rank2", "suit2"]] = df["Hand"].apply(
    lambda h: pd.Series(parse_hand_to_canonical(h))
)

# -------------------------
# 4. Additional Indicators
# -------------------------
df["is_suited"] = (df["suit1"] == df["suit2"]).astype(int)
df["is_pair"]   = (df["rank1"] == df["rank2"]).astype(int)

def is_connector(row):
    return 1 if abs(row["rank1"] - row["rank2"]) == 1 else 0

def is_1_gap(row):
    return 1 if abs(row["rank1"] - row["rank2"]) == 2 else 0

df["is_connector"] = df.apply(is_connector, axis=1)
df["is_1_gap"] = df.apply(is_1_gap, axis=1)

# -------------------------
# 5. Build the Target: Fold Frequency in [0,1]
# -------------------------
df["fold_freq"] = df["FOLD"] / 100.0  # convert from [0..100] to [0..1]

# -------------------------
# 6. Define X (Features) and y (Target)
# -------------------------
feature_cols = [
    "rank1", "suit1", "rank2", "suit2",
    "stack_size",
    "is_suited", "is_pair", "is_connector", "is_1_gap"
]
X = df[feature_cols]
y = df["fold_freq"]

# -------------------------
# 7. K-Fold Cross-Validation
# -------------------------
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)

mse_list = []
for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mse_list.append(mse)

mse_array = np.array(mse_list)
rmse_array = np.sqrt(mse_array)
print("MSE (per fold):", mse_array)
print("RMSE (per fold):", rmse_array)
print("Mean RMSE:", rmse_array.mean(), "Std dev:", rmse_array.std())

# -------------------------
# 8. Train Final Model on ALL Data
# -------------------------
final_model = RandomForestRegressor(n_estimators=100, random_state=42)
final_model.fit(X, y)

# -------------------------
# 9. Prepare Function to Query Any Hand + Stack
# -------------------------
def prepare_features(hand_str, stack_size):
    """
    Convert a hand like 'AcAd' or '2sQd' + a stack size into
    the 9-element feature vector. We do the same canonical parse
    to ensure consistent ordering of the two cards.
    Returns a 2D numpy array suitable for model.predict().
    """
    card1 = hand_str[0:2]  # e.g. '2s'
    card2 = hand_str[2:4]  # e.g. 'Qd'
    
    # Parse & canonicalize
    r1, s1, r2, s2 = parse_hand_to_canonical(hand_str)
    
    # Indicators
    is_suited = 1 if s1 == s2 else 0
    is_pair   = 1 if r1 == r2 else 0
    is_connector = 1 if abs(r1 - r2) == 1 else 0
    is_1_gap = 1 if abs(r1 - r2) == 2 else 0

    features = [
        r1, s1,
        r2, s2,
        stack_size,
        is_suited,
        is_pair,
        is_connector,
        is_1_gap
    ]
    return np.array([features])

# -------------------------
# Example Testing
# -------------------------
test_hands = ["Qd2s", "2sQd", "AcAd", "5h9d", "9h5s"]
stack_size = 90

for hand in test_hands:
    X_custom = prepare_features(hand, stack_size)
    pred_fold = final_model.predict(X_custom)[0]
    pred_raise = 1.0 - pred_fold
    print(f"Hand: {hand}, Stack: {stack_size}")
    print(f"  Predicted fold frequency:  {pred_fold:.4f}  ({pred_fold*100:.2f}%)")
    print(f"  Predicted raise frequency: {pred_raise:.4f} ({pred_raise*100:.2f}%)")
    print("----")


MSE (per fold): [2.62084034e-06 1.19277635e-07 1.97807870e-08 6.79338551e-08
 1.34332828e-07]
RMSE (per fold): [0.0016189  0.00034537 0.00014064 0.00026064 0.00036651]
Mean RMSE: 0.0005464133530296848 Std dev: 0.00054209366008017
Hand: Qd2s, Stack: 90
  Predicted fold frequency:  1.0000  (100.00%)
  Predicted raise frequency: 0.0000 (0.00%)
----
Hand: 2sQd, Stack: 90
  Predicted fold frequency:  1.0000  (100.00%)
  Predicted raise frequency: 0.0000 (0.00%)
----
Hand: AcAd, Stack: 90
  Predicted fold frequency:  0.0000  (0.00%)
  Predicted raise frequency: 1.0000 (100.00%)
----
Hand: 5h9d, Stack: 90
  Predicted fold frequency:  0.2458  (24.58%)
  Predicted raise frequency: 0.7542 (75.42%)
----
Hand: 9h5s, Stack: 90
  Predicted fold frequency:  0.2458  (24.58%)
  Predicted raise frequency: 0.7542 (75.42%)
----


d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
d:\Coding\android_emulator\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor wa